In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import json
import time

In [7]:
url = "https://www.oliveyoung.co.kr/store/store/getStoreListJsonAjax.do?pageIdx=1&searchType=area&tabType=areaTab&rgn1=%EA%B0%95%EC%9B%90%EB%8F%84&rgn2=none&openYn=N&tcCd=&psCd=&usrLat=37.4866415&usrLng=127.0206057"
r = requests.get(url)
bs = BS(r.text)
bs

<html><body><li class="DD41div">
<div class="li_Pc_reInner">
<h4 class="tit">
<a href="javascript:;">춘천온의점</a>
<span class="store_nearWay">74.8km</span>
</h4>
<p class="addr">강원도 춘천시 방송길 77 춘천센트럴타워푸르지오 프리미엄홀 1201,1202호</p>
<div class="area">
<div class="call">033-243-5314</div>
<div class="time on">영업중</div>
</div>
<div class="fv_reShop_in"><span>3,873</span>명이 관심매장으로 등록했습니다.</div>
<button class="star active" onclick="javascript:store.main.setStarEvent($(this))" title="관심매장 해제됨">관심매장</button>
</div>
<input class="strNo" name="strNo" type="hidden" value="DD41"/>
<input class="strNm" name="strNm" type="hidden" value="춘천온의점"/>
<input class="lat" name="lat" type="hidden" value="37.8689954534399"/>
<input class="lng" name="lng" type="hidden" value="127.720699967076"/>
<input class="dist" name="dist" type="hidden" value="74.8"/>
<input class="intCnt" name="intCnt" type="hidden" value="3,873"/>
<input class="favorYn" name="favorYn" type="hidden" value="N"/>
<input class="openYn" name="openYn"

In [10]:
import tqdm

idx = 1
olive_store = []
while True:
    #페이지 계속 바뀌도록 F스트링 사용
    url = f"https://www.oliveyoung.co.kr/store/store/getStoreListJsonAjax.do?pageIdx={idx}&searchType=area&tabType=areaTab&rgn1=none&rgn2=none&openYn=N&tcCd=&psCd=&usrLat=37.4866477&usrLng=127.0206201"
    r = requests.get(url)
    bs = BS(r.text)
    
    if len(bs.findAll("div", class_="li_Pc_reInner")) == 0:
        break
    else:
        olive_soup = []
        
        for i in tqdm.tqdm(range(len(bs.findAll("li")))):
            olive_soup = bs.findAll("li")
            olive_store.append({
            'storeName' : olive_soup[i].find("a").text,
            'address' : olive_soup[i].find("p").text,
            'tel' : olive_soup[i].find("div", {"class":"call"}).text 
            })
    idx += 1

100%|████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 3734.02it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


In [11]:
olive_df = pd.DataFrame(olive_store)
olive_df

,storeName,address,tel
0,남부터미널점,서울특별시 서초구 효령로 283 서경빌딩 1층,02-597-5290
1,교대점,서울특별시 서초구 서초중앙로 95 1층,02-3471-5240
2,서초대로점,서울특별시 서초구 서초대로 314 정보통신공제조합회관,02-521-9094
3,서초유원점,서울특별시 서초구 서초중앙로24길 35,02-537-4685
4,강남우성점,서울특별시 강남구 강남대로 320 황화빌딩 1층 일부,02-501-4426
...,...,...,...
1320,광주첨단점,광주광역시 광산구 첨단중앙로 104,062-971-0625
1321,군포행사장,서울특별시 중구 소월로2길 12,
1322,미아사거리점,서울특별시 강북구 도봉로 41-1,02-983-5290
1323,쌍문역점,서울특별시 도봉구 도봉로 480 1층,02-998-3833


## address 주소 깔끔하게 처리하기

In [12]:
import re
p = re.compile(r'.*(로|길).((\d+)(-)*(\d)*|([지][하] \d*))')
address = re.compile("(([가-힣A-Za-z·\d~\-\.]{2,}(로|길).[\d]+)|([가-힣A-Za-z·\d~\-\.]+(읍|동)\s)[\d]+)")
olive_df['tmp'] = olive_df[olive_df.address.apply(lambda x : len(address.findall(x)) > 0)]['address'].apply(lambda x : address.findall(x))

In [13]:
olive_df[olive_df['tmp'].isnull()]

,storeName,address,tel,tmp
8,양재역점,서울특별시 서초구 남부순환로 지하 2585 신분당선 양재역,02-578-7642,NaN
16,신논현역점,서울특별시 강남구 봉은사로 지하 102 B2,02-569-7390,NaN
21,강남엔터식스점,"서울특별시 서초구 신반포로 지하 188 엔터식스 강남점 26, 27호",02-6321-6104,NaN
42,사당역지하상가점,서울특별시 동작구 동작대로 지하 3,02-521-5390,NaN
168,종로YBM점,서울특별시 종로구 종로 104,02-2285-5290,NaN
173,종각점,서울특별시 종로구 종로 61 1층,02-723-5290,NaN
176,종로1가점,서울특별시 종로구 종로 19 르미에르종로타운 1층,02-720-0918,NaN
183,국회의사당역점,서울특별시 영등포구 국회대로 지하 758 9호선 지하2층,02-784-9698,NaN
209,천호엔터식스점,서울특별시 강동구 천호대로 지하 997 지하1층,02-6484-6110,NaN
295,안양일번가지하상가점,경기도 안양시 만안구 만안로 지하 231 D20호,031-441-5290,NaN


#olive_df.loc[olive_df['address'].str.contains('.*(로|길).((\d+)(-)*(\d)*|([지][하] \d*))'), 'doro_address'] =
olive_df.loc[olive_df['address'].str.contains('.*(로|길).((\d+)(-)*(\d)*|([지][하] \d*))')]['address'].apply(lambda x : " ".join(address.findall(x))) 

#도로명 주소 깔끔하게 전처리 하기 (~~로 XX 형식)
for x in olive_df:
    if olive_df['address'].str.contains('.*(로|길).((\d+)(-)*(\d)*|([지][하] \d*))') == True: #만약 주소 컬럼의 값에 정규식 형태값이 있다면 
        olive_df['doro_address'] = olive_df['address'].str.extract('.*(로|길).((\d+)(-)*(\d)*|([지][하] \d*))') #도로주소 컬럼에 그 정규식 형태값만 빼다 넣기
    else: #해당 정규식 형태값이 없다면 (결측치라면) 
        olive_df['doro_address'] = None #None 값 넣기

In [15]:
olive_df

,storeName,address,tel,tmp
0,남부터미널점,서울특별시 서초구 효령로 283 서경빌딩 1층,02-597-5290,"[(효령로 283, 효령로 283, 로, , )]"
1,교대점,서울특별시 서초구 서초중앙로 95 1층,02-3471-5240,"[(서초중앙로 95, 서초중앙로 95, 로, , )]"
2,서초대로점,서울특별시 서초구 서초대로 314 정보통신공제조합회관,02-521-9094,"[(서초대로 314, 서초대로 314, 로, , )]"
3,서초유원점,서울특별시 서초구 서초중앙로24길 35,02-537-4685,"[(서초중앙로24길 35, 서초중앙로24길 35, 길, , )]"
4,강남우성점,서울특별시 강남구 강남대로 320 황화빌딩 1층 일부,02-501-4426,"[(강남대로 320, 강남대로 320, 로, , )]"
...,...,...,...,...
1320,광주첨단점,광주광역시 광산구 첨단중앙로 104,062-971-0625,"[(첨단중앙로 104, 첨단중앙로 104, 로, , )]"
1321,군포행사장,서울특별시 중구 소월로2길 12,,"[(소월로2길 12, 소월로2길 12, 길, , )]"
1322,미아사거리점,서울특별시 강북구 도봉로 41-1,02-983-5290,"[(도봉로 41, 도봉로 41, 로, , )]"
1323,쌍문역점,서울특별시 도봉구 도봉로 480 1층,02-998-3833,"[(도봉로 480, 도봉로 480, 로, , )]"


In [16]:
for i in olive_df['address']:
    try:
        int(i.split(' ')[-1])
    except Exception as e:
        print(e)

invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '정보통신공제조합회관'
invalid literal for int() with base 10: '일부'
invalid literal for int() with base 10: '양재역'
invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '1~3층'
invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '지하2층'
invalid literal for int() with base 10: 'B2'
invalid literal for int() with base 10: '909-6)'
invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '27호'
invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '102호'
invalid literal for int() with base 10: '106~108호'
invalid literal for int() with base 10: '1층'
invalid literal for int() with base 10: '올리브영'
invalid literal for int() w

In [19]:
olive_df.drop("tmp",axis = 1,inplace=True)

In [20]:
olive_df

,storeName,address,tel
0,남부터미널점,서울특별시 서초구 효령로 283 서경빌딩 1층,02-597-5290
1,교대점,서울특별시 서초구 서초중앙로 95 1층,02-3471-5240
2,서초대로점,서울특별시 서초구 서초대로 314 정보통신공제조합회관,02-521-9094
3,서초유원점,서울특별시 서초구 서초중앙로24길 35,02-537-4685
4,강남우성점,서울특별시 강남구 강남대로 320 황화빌딩 1층 일부,02-501-4426
...,...,...,...
1320,광주첨단점,광주광역시 광산구 첨단중앙로 104,062-971-0625
1321,군포행사장,서울특별시 중구 소월로2길 12,
1322,미아사거리점,서울특별시 강북구 도봉로 41-1,02-983-5290
1323,쌍문역점,서울특별시 도봉구 도봉로 480 1층,02-998-3833


## 데이터 저장

olive_df.to_csv('./OliveYoung.csv')
olive_df.to_json('./OliveYoung.json')
olive_df.to_excel('./OliveYoung.xlsx')

## DB로 데이터 보내기

In [ ]:
import pymysql

def connection(host='encore.cluster-cd9ecw5piy57.ap-northeast-2.rds.amazonaws.com', user='admin', password='newjeans', 
                              charset='utf8', db='encore'):
    
    try:
        con = pymysql.connect(host=host, 
                              user=user, password=password ,
                              charset=charset, db=db)
        cur = con.cursor()
    except Exception as e:
        print ("error ->", e)
    
    return cur, con

In [ ]:
cur, con = connection()

In [ ]:
sql = "INSERT INTO OliveYoung VALUES (%s , %s , %s)"

for idx, row in olive_df.iterrows():
    try:
        cur.execute(sql,list(row.values))
    except:
        pass
    
con.commit()